## 신경망의 구조 : 핵심 Keras API 이해하기

### 층 : 딥러닝의 구성요소
신경망의 기본 데이터 구조는 층(layer)이다. 층은 하나 이상의 텐서를 입력으로 받고 하나 이상의 텐서를 출력하는 데이터 처리 모듈로, 어떤 종류의 층은 상태가 없지만 대부분의 경우(flatten, pooling, dropout 층을 제외하고는) **가중치** 라는 층의 상태를 가지게 된다. 이 가중치는 확률적 경사 하강법으로 학습되는 하나 이상의 텐서이며 여기에는 신경망이 학습한 지식이 담겨있다.<br><br>
층마다 적절한 텐서 포맷과 데이터 처리 방식이 다르다. 예를들어 (samples, features) 크기의 랭크-2 텐서에 저장된 간단한 벡터 데이터는 **밀집 연결층(densely connected layer)**으로 처리하는 경우가 많다(케라스에서는 Dense 클래스에 해당). 이를 **완전 연결층(fully connected layer; fc layer)** 혹은 **밀집 층(dense layer)**라고도 부른다.<br>
(samples, timesteps, features) 크기의 랭크-3 텐서에 저장된 시퀸스 데이터는 일반적으로 LSTM 같은 **순환층(recurrent layer)**이나 1D **합성곱 신경층(convolution layer)** 로 처리한다.

In [139]:
# 케라스에서 Layer의 서브클래스로 구현한 Dense 층
# Layer는 상태(가중치)와 연산9정방향 패스)를 캡슐화한 객체이다.
from tensorflow import keras

# 모든 케라스 층은 Layer 클래스를 상속한다
class Simpledense(keras.layers.Layer) :
  def __init__(self, units, activation=None) :
    super().__init__()
    self.units = units
    self.activation = activation

  # 가중치는 init 메서드에서 만들수도 있지만
  # 일반적으로 build 메서드에서 정의하고, call 메서드에서 연산을 정의한다.
  def build(self, input_shape) :
    input_dim = input_shape[-1]
    # add_weight()는 가중치를 간편하게 만들수 있는 메서드이다.
    # self.W = tf.Variable(tf.random.uniform(w_shape))
    # 위와 같이 독립적으로 변수를 생성하고, 층의 속성으로 할당할 수도 있다.
    self.W = self.add_weight(shape=(input_dim, self.units), initializer='random_normal')
    self.b = self.add_weight(shape=(self.units,), initializer='zeros')

  # call() 메서드에서 정방향 패스 계산을 정의한다
  def call(self, inputs) :
    y = tf.matmul(inputs, self.W) + self.b
    if self.activation is not None :
      y = self.activation(y)
    return y

In [140]:
# 위 클래스의 인스턴스를 생성하면, 텐서플로우 텐서를 입력으로 받는 함수처럼 사용할 수도 있다.
# 앞서 정의한 층의 인스턴스 생성
my_dense = Simpledense(units=32, activation=tf.nn.relu)

# 테스트 입력 생성
input_tensor = tf.ones(shape=(2, 784))

# 이 입력으로 층을 함수처럼 호출한다
output_tensor = my_dense(input_tensor)

# 결과 호출
print(output_tensor)
print(output_tensor.shape)

tf.Tensor(
[[0.0000000e+00 0.0000000e+00 4.0292740e-04 0.0000000e+00 0.0000000e+00
  0.0000000e+00 1.1533540e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  1.4605752e-01 0.0000000e+00 1.4976048e+00 2.2201407e+00 2.7196378e-01
  7.5768644e-01 1.8118950e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 1.7404206e+00 0.0000000e+00 0.0000000e+00 8.6889648e-01
  0.0000000e+00 2.2573924e+00 0.0000000e+00 0.0000000e+00 1.0664822e+00
  0.0000000e+00 1.5510083e+00]
 [0.0000000e+00 0.0000000e+00 4.0292740e-04 0.0000000e+00 0.0000000e+00
  0.0000000e+00 1.1533540e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  1.4605752e-01 0.0000000e+00 1.4976048e+00 2.2201407e+00 2.7196378e-01
  7.5768644e-01 1.8118950e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 1.7404206e+00 0.0000000e+00 0.0000000e+00 8.6889648e-01
  0.0000000e+00 2.2573924e+00 0.0000000e+00 0.0000000e+00 1.0664822e+00
  0.0000000e+00 1.5510083e+00]], shape=(2, 32), dtype=float32)
(2, 32)


### 케라스의 층 호환
**층 호환(layer compatibility)** 개념은 모든 층이 특정 크기의 입력 텐서만 받고, 특정 크기의 출력 텐서를 반환한다는 사실을 의미한다. 케라스를 사용할 때는 대부분의 경우 크기 호환성에 대해 걱정할 필요가 없다. 모델에 추가하는 층은 앞선 층의 크기에 맞도록 동적으로 만들어 지기 때문인데, 예를 들어
```python
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential([
  layers.Dense(32, activation='relu'),
  layers.Dense(32)
])
```
위 모델은 입력 크기에 대해 어떠한 정보도 받지 않는다. 그 대신 입력 크기를 처음 본 입력의 크기로 추론한다.
<br><br>
Layer 클래스의 `__call__()` 메서드가 자동 크기 추론만 처리하는 것은 아니다. 사용자 정의 층을 구현할 때 call() 메서드에 정방향 패스 계산을 넣으며, 즉시 실행과 그래프 실행(graph execution) 사이를 전환하고 입력 마스킹(masking)을 처리하는 등 더 많은 작업을 관리한다.

### 층에서 모델로
딥려닝 모델은 층으로 구성된 그래프이다. 케라스에서는 Model 클래스에 해당하는데, 앞서 많이 본 Sequential 모델의 경우 단순히 층을 쌓은 것으로 하나의 입력을 하나의 출력에 매핑한다. 앞으로는 매우 다양한 종류의 네트워크를 보게 될 것인데 그 중 자주 등장하는 구조로는
- 2개의 가지(two-branch)를 가진 네트워크
- 멀티헤드(multihead) 네트워크
- 잔차 연결(residual connection)
등이 있다. 네트워크 구조(topology)는 꽤나 복잡할 수도 있는데, 케라스에서 이러한 모델을 만드는 방법은 일반적으로 두 가지다. 직접 모델 클래스의 서브클래스를 만들거나, 더 적은 코드로 많은 일을 수행할 수 있는 함수형 API를 사용하는 것이다.<br><br>
오델의 구조는 **가설 공간(hypothesis space)** 를 정의한다. 네트워크 구조를 선택하면 가능성 있는 공간(가설 공간)이 입력 데이터를 출력 데이터로 매핑하는 일련의 특정한 텐서 연산으로 제한되는데, 그 다음 우리가 찾을 것은이런 텐서 연산에 관련된 가중치 텐서의 좋은 값이다.<br><br>
데이터에서 학습을 하려면 데이터에 대한 가정을 해야하고, 이런 가정이 학습할 수 있는 것을 정의해야한다. 따라서 가설 공간의 구조, 즉 모델의 구조는 매우 중요하다. 문제에 대한 가정, 즉 시작할 때 모델이 가지게 될 사전지식을 인코딩 한다.

### 컴파일 : 학습 과정설명
- 손실함수(loss function; 목적함수 objective function) : 훈련 과정에서 최소화할 값. 현재 작업에 대한 성공의 척도
- 옵티마이저 : 손실함수를 기반으로 네트워크가 어떻게 업데이트 될지 결정한다. 특정 종류의 확률적 경사 하강법(SGD)로 구현
- 측정 지표(metric) : 훈련 검증과정에서 모니터링할 성공의 척도(예를 들어 분류 정확도) 손실과 달리 훈련은 측정 지표에 직접 최적화되지 않는다. 따라서 측정 지표는 미준 가능하지 않아도 괜찮다.

In [141]:
x = np.arange(1, 31)
x

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])

In [142]:
# train:validation = 8:2로 나누고 섞여있어야 한다.
from sklearn.model_selection import train_test_split

np.random.shuffle(x)

train, val = train_test_split(x, test_size=0.2)

In [143]:
print(train.shape, val.shape)
print(train)
print(val)

(24,) (6,)
[ 8 27  3 30 17 28 24 10 19 14  6 21 13 23  4 18  7  5 16 12 15 29 25 26]
[ 9 20 11  1  2 22]


### fit() 메서드
compile() 다음에는 fit() 메서드를 호출한다. 이 메서드는 훈련 루프를 구현하는데, 주요 매개변수는 아래와 같다.
- 훈련할 데이터(입력과 타깃) : 일반적으로 넘파이 배열이나 텐서플로 Dataset 객체로 전달한다.
- 훈련할 에폭(epoch) 횟수 : 전달한 데이터에서 훈련 루프를 몇 번이나 반복할지 알려준다
- 미니 배치 경사 하강법의 각 에폭에서 사용할 배치 크기 : 가중치 업데이트 단계에서 그레이디언트를 계산하는데 사용될 훈련 샘플 개수를 말한다.<br><br>
fit()을 호출하면 History 객체가 반환된다. 이 객체는 딕셔너리인 history 속성을 가지고 있는데, loss 또는 특정 지표 이름의 키와 각 에포크 값의 리스트를 매핑한다.